In [14]:
# %load LDA-Copy1.py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import ast
import util
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
import gensim
import pyLDAvis.gensim
import pickle

In [15]:
# df = util.get_processed_data("./data/collections_all_science_out-temp.csv", True)
df = pd.read_csv("./Final_Data/collections_all_science_out-temp_lda.csv")

In [16]:
df.head(1)

,resource_id,title,url,created_at,updated_at,creator_id,modifier_id,original_creator_id,original_content_id,parent_content_id,...,creator_system,tenant,tenant_root,primary_language,max_score,Summarization,text,clean,clean_matrix,lda_topics
0,9dad3a1d-6f8b-4ef6-bc2c-b207d8d2ce1f,Table of Contents - Google Docs,https://docs.google.com/document/d/1e9X9BmWycf...,2014-09-23 03:16:24,2016-11-02 23:13:58.325,f3495a12-a0f4-4092-8cba-6817e9818f18,f3495a12-a0f4-4092-8cba-6817e9818f18,f3495a12-a0f4-4092-8cba-6817e9818f18,0c47ef6b-cdc1-45bc-8764-476f0d422566,0c47ef6b-cdc1-45bc-8764-476f0d422566,...,Gooru-2.0-v2,ba956a97-ae15-11e5-a302-f8a963065976,NaN,1,NaN,"Atom FCNElement C, M WSChemical Rxn FCNChem an...",Table Contents Google Docs Table Contents Goog...,"['atom', 'fcnelement', 'c', 'wschemical', 'rxn...","[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1...","[(0, 0.0029411765), (1, 0.0029411765), (2, 0.0..."


In [17]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

In [18]:
df.Summarization.replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)
df['clean'] = df.Summarization.apply(clean)
df.clean = df.clean.apply(lambda x: x.split())
dictionary = gensim.corpora.Dictionary(df.clean)
df['clean_matrix'] = df.clean.apply(lambda x: dictionary.doc2bow(x))


file_Name = './Final_Data/lda_science_dictionary.model'
fileObject = open(file_Name,'wb') 

pickle.dump(dictionary,fileObject)   

fileObject.close()


In [19]:
lda = gensim.models.ldamodel.LdaModel
np.random.seed(1)
ldamodel = lda(
    df.clean_matrix, 
    num_topics=20, 
    id2word = dictionary, 
    passes=100,
    minimum_probability=0.0)

In [20]:
for idx, topic in ldamodel.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.052*"matter" + 0.021*"atom" + 0.018*"text" + 0.017*"law" + 0.017*"made" + 0.016*"1" + 0.014*"2" + 0.013*"force" + 0.012*"explains" + 0.012*"acceleration"
Topic: 1 
Words: 0.008*"land" + 0.007*"new" + 0.005*"national" + 0.005*"world" + 0.005*"place" + 0.005*"water" + 0.004*"first" + 0.004*"east" + 0.004*"bitesizeall" + 0.004*"ie"
Topic: 2 
Words: 0.140*"grade" + 0.030*"35" + 0.029*"912" + 0.028*"k2" + 0.028*"68" + 0.028*"collegeadult" + 0.022*"discipline" + 0.021*"rock" + 0.021*"page" + 0.016*"language"
Topic: 3 
Words: 0.017*"concept" + 0.012*"1" + 0.011*"review" + 0.011*"party" + 0.010*"see" + 0.009*"term" + 0.009*"page" + 0.008*"card" + 0.008*"political" + 0.008*"definition"
Topic: 4 
Words: 0.015*"atom" + 0.013*"electron" + 0.012*"energy" + 0.010*"chemical" + 0.010*"element" + 0.008*"number" + 0.007*"atomic" + 0.006*"text" + 0.006*"form" + 0.006*"read"
Topic: 5 
Words: 0.019*"earth" + 0.017*"energy" + 0.015*"water" + 0.010*"change" + 0.009*"force" + 0.008*"read" +

In [21]:
# df['lda_topic_index'] = df.clean_matrix.apply(lambda x: sorted(ldamodel[x], key=lambda tup: -1*tup[1])[0][0])

# df['lda_topic'] = df.lda_topic_index.apply(lambda x: ldamodel.print_topic(x, 10))

# for index, score in sorted(ldamodel[df.clean_matrix[1]], key=lambda tup: -1*tup[1]):
#     print("\nScore: {}\t \nTopic: {}".format(score, ldamodel.print_topic(index, 20)))



In [22]:
vis_data = pyLDAvis.gensim.prepare(ldamodel,df.clean_matrix,dictionary)

pyLDAvis.display(vis_data)

/home/bukka/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [23]:
ldamodel.save('./Final_Data/lda_science.model')
# ldamodel = ldamodel.load('./lda_25_sc/lda_25_sc.model')

In [ ]:
# doc_topics = ldamodel.get_document_topics(df.clean_matrix[1],per_word_topics=True)[0]

df['lda_topics'] = df.clean_matrix.apply(lambda x: ldamodel.get_document_topics(x,per_word_topics=True)[0])
df['lda_topics'] = df.lda_topics.apply(lambda x : [b for _ ,b in ast.literal_eval(x) ])

# df.to_csv('./data/collections_all_science_out-temp.csv')

In [ ]:
df.lda_topics[0]

In [ ]:
df.head()

In [ ]:
df.to_csv("./data/collections_all_science_out-temp_lda.csv",index=False)

In [ ]:
df.shape